In [93]:
import codecs
import re
import pandas as pd
import numpy as np
from numpy import log,min

In [94]:
data_path = r'E:\data\NLP\book\bible.txt'
f = codecs.open(data_path,'r','utf-8')
s = f.read()
s = re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）()“”：‘’；:]+", "",s)
s = re.sub("[a-zA-Z0-9]","",s)

In [95]:
s = s[1:]
s[:3000]

'创世记起初神创造天地地是空虚混沌渊面黑暗神的灵运行在水面上神说要有光就有了光神看光是好的就把光暗分开了神称光为昼称暗为夜有晚上有早晨这是头一日神说诸水之间要有空气将水分为上下神就造出空气将空气以下的水空气以上的水分开了事就这样成了神称空气为天有晚上有早晨是第二日神说天下的水要聚在一处使旱地露出来事就这样成了神称旱地为地称水的聚处为海神看著是好的神说地要发生青草和结种子的菜蔬并结果子的树木各从其类果子都包著核事就这样成了于是地发生了青草和结种子的菜蔬各从其类并结果子的树木各从其类果子都包著核神看著是好的有晚上有早晨是第三日神说天上要有光体可以分昼夜作记号定节令日子年岁并要发光在天空普照在地上事就这样成了于是神造了两个大光大的管昼小的管夜又造众星就把这些光摆列在天空普照在地上管理昼夜分别明暗神看著是好的有晚上有早晨是第四日神说水要多多滋生有生命的物要有雀鸟飞在地面以上天空之中神就造出大鱼和水中所滋生各样有生命的动物各从其类又造出各样飞鸟各从其类神看著是好的神就赐福给这一切说滋生繁多充满海中的水雀鸟也要多生在地上有晚上有早晨是第五日神说地要生出活物来各从其类牲畜昆虫野兽各从其类事就这样成了于是神造出野兽各从其类牲畜各从其类地上一切昆虫各从其类神看著是好的神说我们要照著我们的形像按著我们的样式造人使他们管理海里的鱼空中的鸟地上的牲畜和全地并地上所爬的一切昆虫神就照著自己的形像造人乃是照著他的形像造男造女神就赐福给他们又对他们说要生养众多遍满地面治理这地也要管理海里的鱼空中的鸟和地上各样行动的活物神说看哪我将遍地上一切结种子的菜蔬和一切树上所结有核的果子全赐给你们作食物至于地上的走兽和空中的飞鸟并各样爬在地上有生命的物我将青草赐给它们作食物事就这样成了神看著一切所造的都甚好有晚上有早晨是第六日天地万物都造齐了到第七日神造物的工已经完毕就在第七日歇了他一切的工安息了神赐福给第七日定为圣日因为在这日神歇了他一切创造的工就安息了创造天地的来历在耶和华神造天地的日子乃是这样野地还没有草木田间的菜蔬还没有长起来因为耶和华神还没有降雨在地上也没有人耕地但有雾气从地上腾滋润遍地耶和华神用地上的尘土造人将生气吹在他鼻孔里他就成了有灵的活人名叫亚当耶和华神在东方的伊甸立了一个园子把所造的人安置在那里耶和华神使各样的树从地里长出来可以悦人的眼目其上的果子好作食物园子当中又有生命树和分别善恶的树

In [96]:
#为了方便调用，自定义了一个正则表达式的词典
myre = {2:'(..)', 3:'(...)', 4:'(....)', 5:'(.....)', 6:'(......)', 7:'(.......)'}

min_count = 10 #录取词语最小出现次数
min_support = 30 #录取词语最低聚合度，1代表着随机组合
min_s = 3 #录取词语最低信息熵，越大说明越有可能独立成词
max_sep = 4 #候选词语的最大字数
t=[] #保存结果用。

t.append(pd.Series(list(s)).value_counts()) #逐字统计
tsum = t[0].sum() #统计总字数
rt = [] #保存结果用

for m in range(2, max_sep+1):
    print(u'正在生成%s字词...'%m)
    t.append([])
    for i in range(m): #生成所有可能的m字词
        t[m-1] = t[m-1] + re.findall(myre[m], s[i:])
    
    t[m-1] = pd.Series(t[m-1]).value_counts() #逐词统计
    t[m-1] = t[m-1][t[m-1] > min_count] #最小次数筛选
    tt = t[m-1]
    for k in range(m-1):
        qq = np.array(list(map(lambda ms: tsum*t[m-1][ms]/t[m-2-k][ms[:m-1-k]]/t[k][ms[m-1-k:]], tt.index))) > min_support #最小聚合度筛选。
        tt = tt[qq]
    rt.append(tt.index)
#     print(tt.index[:10])
def cal_S(sl): #信息熵计算函数
    return -((sl/sl.sum()).apply(log)*sl/sl.sum()).sum()

正在生成2字词...
正在生成3字词...
正在生成4字词...


In [97]:
for i in range(2, max_sep+1):
    print(u'正在进行%s字词的最大熵筛选(%s)...'%(i, len(rt[i-2])))
    pp = [] #保存所有的左右邻结果
    for j in range(i+2):
        pp = pp + re.findall('(.)%s(.)'%myre[i], s[j:])
    pp = pd.DataFrame(pp).set_index(1).sort_index() #先排序，这个很重要，可以加快检索速度
    index = np.sort(np.intersect1d(rt[i-2], pp.index)) #作交集
    #下面两句分别是左邻和右邻信息熵筛选
    index = index[np.array(list(map(lambda s: cal_S(pd.Series(pp[0][s]).value_counts()), index))) > min_s]
    rt[i-2] = index[np.array(list(map(lambda s: cal_S(pd.Series(pp[2][s]).value_counts()), index))) > min_s]

正在进行2字词的最大熵筛选(3226)...
正在进行3字词的最大熵筛选(2354)...
正在进行4字词的最大熵筛选(1780)...


In [98]:
#下面都是输出前处理
for i in range(len(rt)):
    t[i+1] = t[i+1][rt[i]]
    t[i+1] = t[i+1].sort_values(ascending = False)

In [99]:
res_path = r'E:\data\NLP\book\result.txt'
# res = pd.DataFrame({'2':t[1].index,'2_num':t[1].values,'3':t[2].index,'3_num':t[2].values,'4':t[3].index,'4_num':t[3].values})
res = pd.DataFrame()
for i in range(2,max_sep+1):
    id = str(i)
    num = id+'num'
    res[id] = t[i-1].index[:20]
    res[num] = t[i-1].values[:20]

In [100]:
res

,2,2num,3,3num,4,4num
0,因为,2449,耶和华,7108,耶路撒冷,828
1,没有,1775,所罗门,318,亚伯拉罕,263
2,耶稣,1639,注或作,313,注原文作,256
3,自己,1522,利未人,308,非利士人,251
4,那里,1514,回答说,270,所说的话,107
5,一切,1451,以利亚,255,耶罗波安,103
6,一个,1230,约书亚,241,法利赛人,102
7,大卫,1164,外邦人,210,在你面前,98
8,不可,1131,那里去,199,直到永远,94
9,知道,1102,犹太人,198,在我面前,89
